# Bedrock AgentCore Memory 検証: トラブルシューティング日報編 (修正完了版)

「エンジニアがよくやるミス（トラブルシューティング）」を題材に、Episode機能の有無による**エージェントの対応の違い**を検証します。

## 修正ポイント
前回エラーになった `list_sessions` の代わりに、**Namespaceを明示した `list_items`** を使用して確実に記憶（要約）を取得します。

## 検証シナリオ
1.  **Episode 1 (過去)**: ユーザー「Lambdaが動かない。原因は **環境変数の設定漏れ** だった。」
2.  **Episode 2 (現在)**: ユーザー「またLambdaが動かないんだよね〜」

---

In [1]:
# 1. 必要なライブラリのインポート
import uuid
import time
import json
import boto3
from datetime import datetime

try:
    from bedrock_agentcore.memory import MemoryClient
    print("Successfully imported MemoryClient.")
except ImportError:
    print("Installing bedrock-agentcore...")
    !pip install bedrock-agentcore strands-agents strands-agents-tools --quiet
    from bedrock_agentcore.memory import MemoryClient

# クライアント初期化
REGION = "us-east-1" 
memory_client = MemoryClient(region_name=REGION)
bedrock_runtime = boto3.client('bedrock-runtime', region_name=REGION)

ACTOR_ID = "engineer_tanaka_001"

Successfully imported MemoryClient.


## 2. メモリリソース作成関数
Episode機能のON/OFFを切り替えてメモリを作成します。

In [3]:
def create_memory_resource(enable_episode=False):
    unique_id = str(uuid.uuid4())[:6]
    memory_name = f"trouble_mem_{unique_id}"
    
    # Semantic Memory (事実の記憶)
    strategies = [
        {
            "semanticMemoryStrategy": {
                "name": "factStrategy",
                "namespaces": ["/strategies/{memoryStrategyId}/actors/{actorId}/facts"]
            }
        }
    ]
    
    # Episode機能のON/OFF
    if enable_episode:
        strategies.append({
            "episodicMemoryStrategy": {
                "name": "episodeStrategy",
                # エピソード（セッションごとの要約）
                "namespaces": ["/strategies/{memoryStrategyId}/actors/{actorId}/sessions/{sessionId}"],
                # Reflection（洞察）: エピソードの親階層に設定する必要あり
                "reflectionConfiguration": {
                     "namespaces": ["/strategies/{memoryStrategyId}/actors/{actorId}"]
                }
            }
        })
        label = "[Episode ON]"
    else:
        label = "[Episode OFF]"

    print(f"Creating Memory {label}: {memory_name} ...")
    memory_info = memory_client.create_memory_and_wait(
        name=memory_name,
        description=f"Troubleshooting memory {label}",
        strategies=strategies
    )
    return memory_info['id']

# 2つのメモリを作成
mem_id_off = create_memory_resource(enable_episode=False)
mem_id_on = create_memory_resource(enable_episode=True)

Creating Memory [Episode OFF]: trouble_mem_192dc0 ...
Creating Memory [Episode ON]: trouble_mem_e49897 ...


## 3. 過去の日報データの投入
「環境変数忘れ」の失敗談を注入します。

In [4]:
def inject_trouble_reports(memory_id):
    # --- Episode 1: 過去の失敗 ---
    session_id_1 = f"daily_report_20241001_{str(uuid.uuid4())[:4]}"
    messages_1 = [
        ("今日の日報です。AWS Lambdaの実装で3時間もハマってしまいました。", "USER"),
        ("それは大変でしたね。どのようなエラーでしたか？", "ASSISTANT"),
        ("実行時にキーエラーが出て動かなかったんです。コードは合ってるはずなのに。", "USER"),
        ("原因は特定できましたか？", "ASSISTANT"),
        ("はい。結局、Lambdaのコンソール画面で環境変数 `DB_CONNECTION_STRING` を設定し忘れてたのが原因でした。", "USER"),
        ("なるほど、環境変数の設定漏れはよくあるミスですね。記録しておきます。", "ASSISTANT")
    ]
    
    print(f"Injecting Episode 1 (Env Var Failure) -> {memory_id} ...")
    memory_client.create_event(
        memory_id=memory_id,
        actor_id=ACTOR_ID,
        session_id=session_id_1,
        messages=messages_1
    )
    print("Done.")

print("--- Processing Episode OFF Memory ---")
inject_trouble_reports(mem_id_off)

print("\n--- Processing Episode ON Memory ---")
inject_trouble_reports(mem_id_on)

--- Processing Episode OFF Memory ---
Injecting Episode 1 (Env Var Failure) -> trouble_mem_192dc0-33owL337EE ...
Done.

--- Processing Episode ON Memory ---
Injecting Episode 1 (Env Var Failure) -> trouble_mem_e49897-IV8sWxCK4B ...
Done.


## 4. 記憶の取得とエージェント回答

ここで `list_items` を使用して、正しいNamespaceから記憶を取得します。
処理完了まで約60秒待ちます...

In [19]:
print("Waiting for AgentCore processing (sleeping 60s)...")
time.sleep(60)
agent_runtime_client = boto3.client('bedrock-agent-runtime', region_name=REGION)

def get_memory_context(memory_id):
    """
    メモリ内の情報を取得してテキスト化するヘルパー関数
    """
    context_text = ""
    
    try:
        # 1. メモリ設定の取得
        mem_details = memory_client.get_memory(memoryId=memory_id)
        memory_data = mem_details.get('memory', {})
        strategies = memory_data.get('strategies', [])
        
        target_strategy_id = None
        
        # Strategyの特定
        for s in strategies:
            # nameかtypeで特定
            if s.get('name') == 'episodeStrategy' or 'episodicMemoryStrategy' in s:
                target_strategy_id = s.get('strategyId')
                break 

        # --- A. Episode Strategyがある場合 ---
        if target_strategy_id:
            # 検索対象のNamespace
            # ここでは「過去の全セッション」から検索したいため、sessionsまでを指定します
            namespace = f"/strategies/{target_strategy_id}/actors/{ACTOR_ID}/sessions"
            
            print(f"DEBUG: Querying memories via SDK from: {namespace}")

            try:
                results = memory_client.retrieve_memories(
                    memory_id=memory_id,
                    namespace=namespace,
                    query="error failure Lambda trouble", # 失敗談を引っ掛けるための緩いクエリ
                    top_k=10
                )
                
                if results:
                    print(f"  [Episode ON] {len(results)}個のエピソードが見つかりました")
                    for item in results:
                        # 構造: item['content']['text'] に要約が入ります
                        content = item.get('content', {})
                        text = content.get('text')
                        
                        # 念のためmetadataも確認（いつの記憶かなど）
                        timestamp = item.get('memoryTime', '')
                        
                        if text:
                            context_text += f"- 過去の記憶({timestamp}): {text}\n"
                else:
                    print("  [Episode ON] エピソードは見つかりませんでした (ヒットなし)")
                    
            except Exception as e:
                print(f"  [Warning] retrieve_memories failed: {e}")
                
        else:
            print("  [Warning] Episode Strategyの設定が見つかりませんでした")

    except Exception as e:
        print(f"  Error fetching memory: {e}")
        
    if not context_text:
        context_text = "(特に関連する記憶はありません)"
        
    return context_text

def ask_agent(memory_id, user_query, title):
    print(f"\n{'='*60}")
    print(f" {title}")
    print(f"{'='*60}")

    # 1. 記憶を取得
    context = get_memory_context(memory_id)
    
    # 2. プロンプト作成
    prompt = f"""
あなたは優秀なシニアエンジニアのアシスタントです。
以下の【過去の記憶】を参考にして、ユーザーの困りごとに対してアドバイスをしてください。
もし記憶の中に、今回のトラブルの原因になりそうな「過去の失敗パターン」があれば、それを1番はじめに、具体的に指摘してください。

【過去の記憶】
{context}

ユーザー: {user_query}
AI:"""

    # 3. Amazon Nova Micro で回答生成
    try:
        model_id = "amazon.nova-micro-v1:0"
        body = json.dumps({
            "inferenceConfig": {"max_new_tokens": 512},
            "messages": [
                {"role": "user", "content": [{"text": prompt}]}
            ]
        })

        response = bedrock_runtime.invoke_model(modelId=model_id, body=body)
        response_body = json.loads(response.get('body').read())
        ai_response = response_body['output']['message']['content'][0]['text']
        
        print(f"\n  [🤖 AIエージェントの回答 (Nova Micro)]")
        print(f"  {ai_response}")
        
    except Exception as e:
        print(f"  Invoke Error: {e}")


# 検証実行！
query = "Lambdaが動かないんだよね〜なんでだろう"

ask_agent(mem_id_off, query, "❌ Episode OFF の場合")
ask_agent(mem_id_on, query, "✅ Episode ON の場合")

Waiting for AgentCore processing (sleeping 60s)...

 ❌ Episode OFF の場合
  [Warning] Episode Strategyの設定が見つかりませんでした

  [🤖 AIエージェントの回答 (Nova Micro)]
  Lambdaが動かない問題に直面しているのですね。それでは、まずはトラブルの原因を特定するためのステップを提案します。また、過去の失敗パターンが明らかに関連していると判断できないため、一般的なトラブルシューティングのポイントから始めましょう。

### トラブルシューティングのステップ

1. **ログの確認**
   - AWS Lambdaのログを確認してください。CloudWatch Logsで関数の詳細なエラーメッセージや実行ログを確認できます。これにより、関数が失敗した具体的な原因が分かることが多いです。

2. **タイムアウト設定**
   - Lambda関数のタイムアウト設定が適切か確認してください。関数の実行時間がタイムアウト値を超えている場合、Lambdaは実行を停止します。この場合、関数の実行時間を増やすか、コードの効率を向上させる必要があります。

3. **権限設定**
   - Lambda関数のIAMロールに適切な権限が付与されているか確認してください。必要なリソースにアクセスするためには適切な権限が必要です。

4. **依存ライブラリの確認**
   - Lambda関数が外部ライブラリやサービスに依存している場合、それらが正しく機能しているか確認してください。例えば、S3バケット、DynamoDBテーブル、API Gatewayなど。

5. **環境変数の確認**
   - Lambda関数に環境変数が設定されている場合、それらの値が正しいか確認してください。誤った環境変数の値は関数の動作に影響を与える可能性があります。

### 過去の失敗パターン

【過去の記憶】には特に関連する失敗パターンが見当たりませんが、一般的には以下のようなパターンがよく見られます。

- **依存ライブラリのバージョン不一致**：Lambda関数が使用するライブラリのバージョンが期待と異なる場合、関数が予期せず動作しなくなる可能性があります。
- **IA

## 結果の見方

1.  **Episode OFF**: 過去の失敗（環境変数）が「Fact（事実）」として断片的に残っているかもしれませんが、文脈としては弱いため、AIは「ログを見ろ」「権限を確認しろ」といった**一般的なトラブルシューティング**を回答する傾向があります。

2.  **Episode ON**: 過去のセッションが「要約」として残っているため、AIは「以前、環境変数の設定漏れでエラーになっていましたね」と**ピンポイントで指摘**できます。

In [ ]:
# クリーンアップ
memory_client.delete_memory(memory_id=mem_id_off)
memory_client.delete_memory(memory_id=mem_id_on)
print("Cleanup complete.")